###     Instructions to execute
#### 1) Start with the IDS561_twitter_app, execute the cells in the order in which they are witten.  Once the 
####     fourth cell is excuted, the app waits for the IDS561_spark_app to connect with the socket (127.0.0.1,5001)
####     displaying the message "Waiting for TCP Connection"
#### 2) Now start exceuting the cells of IDS561_spark_app in the order in which ther are written. Once the cell    
####     containing ssc.start() is executed, the connection is formed and the tweets are extracted by the api.
####     ssc.stop() breaks the connection
#### 3) As the tweets are being extracted, they are appended into the list. The list is converted into DataFrame and 
####     that DataFrame is outputted as .csv.
#### 4) Later that csv is imported into Sentiment_score_calculation. Where the sentiment score is calculated. 

In [6]:
import socket
import sys
import requests
import requests_oauthlib
import json
import tweepy
import pandas as pd

In [7]:
# Entering Twitter-API credentials
access_token = '1294388428575047689-uQDkJfSiCikiKi1Yi25NvoFh1SKPvT'
access_token_secret = 'FWBHK9tgHXUzTvIljhwOrCbBn1aIrOD9VLNGigUXbRBzg'
consumer_key = 'HhzYCY5pUaJII2gUAvzqSVyX3'
consumer_secret = 'UqbEdCradZDBEvWCxQCNTwjhJ1szw10iv9JVHmuaAjSZMR49yl'

In [9]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [18]:
TCP_IP = "127.0.0.1" # Setting up the IP at which the twitter app will listen
TCP_PORT = 5001 # Setting up the port number at which the twitter app will listen
conn = None
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM) # Creating a socket 
s.bind((TCP_IP, TCP_PORT))
s.listen(1)
print("Waiting for TCP connection...")
conn, addr = s.accept()
print("Connected... Starting getting tweets.")
tweets = [] 
# Using tweepy Cursor getting, tweets with keyword #hospital 
hospital_tweet = tweepy.Cursor(api.search, q='#hospital',lang = 'en').items(5000)  
for tweet in hospital_tweet:
    tweets.append(tweet.text)
    print(tweet.text)
    # After getting the tweets sending it to the spark app.
    conn.send( tweet.text.encode('utf-8') )

Waiting for TCP connection...
Connected... Starting getting tweets.
RT @NewsRajwar: The adept #team of #doctors to include the Regimental Medical #Officer  and doctors from #District #Hospital ,Handwara  wer…
RT @SDiaso: If Any #Protester Who Was #Shoot By The Combined Islamist #Fulani Nigeria #Military, &amp; Is Taken To Any #Hospital That Happened…
RT @okhistory: OHS photo of the week

Utopia Hospital at 415 NE 1st Street in Oklahoma City, c. 1908
This is believed to be the first Black…
RT @Design4agedcare: Dementia care tailored to the patient makes a difference: Survey https://t.co/YtsJMWW4hu #dementia #alzheimers #hospit…
RT @itskabhijeet: #Urgent #Bangalore #Oct21
#O+ve/Any blood donors
#3 year old child #Hospital- Aster CMI hospital, new airport road, after…
RT @SDiaso: If Any #Protester Who Was #Shoot By The Combined Islamist #Fulani Nigeria #Military, &amp; Is Taken To Any #Hospital That Happened…
RT @itskabhijeet: #Urgent #Bangalore #Oct21
#O+ve/Any blood donors
#3 year old c

In [26]:
tweets

['RT @BarbilCS: Poor &amp; worst medical #FacilitiesManagement @DistAdmKeonjhar exposed in minerals rich #ActualMiningAffected @Jodamhud @DDMJODA…',
 "Beijing Speaks Out about Chinese and Taiwan's Foreign Affairs Department... https://t.co/pqAy6XpJaI via @YouTube… https://t.co/921KEj6tfj",
 'FK065-36B Swirl ceiling diffuser\n#Hospital #Medical #Room #MedicalBed #BedHead #HeadUnit #Aluminum #damper… https://t.co/VUltwtnN3C',
 'Woman found alive in #Detroit funeral home later dies in #hospital https://t.co/o9JNO9exFp via @Yahoo #Michigan #horror #lawsuit',
 'RT @AilishSinclair: “Despite the fact that I wrote of truly terrible events involving that steeple, I found it somewhat comforting to see i…',
 '#Secure your #financial #savings from #hospital #expenses. Click here https://t.co/GavqXf1cWF to know more. https://t.co/itgjo7ejNN',
 'While the majority get #medical treatment and survive, roughly a third #succumb at home or at work without ever mak… https://t.co/tVNeZNdCoG',
 'RT @djgeniu

In [83]:
# Transfering the tweets into the Dataframe
x = pd.DataFrame(tweets)

In [84]:
# To print all the rows of the Dataframe
pd.options.display.max_rows = 5000

In [102]:
# Exporting the tweets as a csv
x.to_csv('test1.csv',index = False)